In [ ]:
import os
from IPython.display import display
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pathlib
import tensorflow as tf
import cv2


# Set the paths for the image, saved model, and annotations

IMAGE_PATHS = ''  # Path to your input image
PATH_TO_MODEL_DIR = ''  # Path to your saved model directory
PATH_TO_LABELS = ''  # Path to your label map file
MIN_CONF_THRESH = float(0.60)

# Import necessary modules

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

# Load the saved model
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

# Create the category index from the label map file

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

image = cv2.imread(IMAGE_PATHS)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Expand dimensions of the image for the model input

image_expanded = np.expand_dims(image_rgb, axis=0)
input_tensor = tf.convert_to_tensor(image)
input_tensor = input_tensor[tf.newaxis, ...]

# Perform object detection on the input image

detections = detect_fn(input_tensor)
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}
detections['num_detections'] = num_detections
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
image_with_detections = image.copy()
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=0.5,
      agnostic_mode=False,
      line_thickness=2)
# cv2_imshow(image_with_detections)
display(Image.fromarray(image_with_detections))